In [1]:
import importlib
import sys, os
sys.path.append('C:\\Users\\bartm\\Documents\\These\\phyloreplica\\src')
from PhyloDataset import *

from PhyloTrees import *

from vae import *

import torch
torch.autograd.set_detect_anomaly(True)

datapath1 = "../data/PF00072/PF00072_rp15_has_PF00196.faa"
datapath2 = "../data/PF00072/PF00072_rp15_has_PF00486.faa"
datapath3 = "../data/PF00072/PF00072_rp15_has_PF00512.faa"
datapath4 = "data/PF00072/PF00072_rp15_has_PF00158.faa"
datapath5 = "data/PF00072/PF00072_rp15_has_PF00990.faa"
datapath6 = "data/PF00072/PF00072_rp15_has_PF01339.faa"
datapath7 = "data/PF00072/PF00072_rp15_has_PF04397.faa"
datapath8 = "data/PF00072/PF00072_rp15_has_PF12833.faa"

lossfn = vae_loss

dataset1 = MSA(datapath1)
dataset2 = MSA(datapath2)
dataset3 = MSA(datapath3)
lt = len(dataset1) + len(dataset2) + len(dataset1)
l1 = int(32*len(dataset1)/lt) 
l2 = int(32*len(dataset2)/lt) 
l3 = 32 - l1 -l2

vae1 = VAE(21, 5, dataset1.len_protein * dataset1.q, [512, 256, 128])
optimizer1 = optim.Adam(vae1.parameters(),weight_decay=0.01)
Node1 = PhyloNode(vae1,
          optimizer1, 
          lossfn,
          parent=None, 
          children=[], 
          dataset = dataset1, 
          tuplesize=2, 
          batch_size=l1, 
          gammaManager = gammaManager_Independant(),
          Name = "196"
    )


vae2 = VAE(21, 5, dataset2.len_protein * dataset2.q, [512, 256, 128])
optimizer2 = optim.Adam(vae2.parameters(),weight_decay=0.01)
Node2 = PhyloNode(vae2,
          optimizer2, 
          lossfn,
          parent=None, 
          children=[], 
          dataset = dataset2, 
          tuplesize=2, 
          batch_size=l2, 
          gammaManager = gammaManager_Independant(),
          Name="486"
    )


vae3 = VAE(21, 5, dataset3.len_protein * dataset3.q, [512, 256, 128])
optimizer3 = optim.Adam(vae3.parameters(),weight_decay=0.01)
Node3 = PhyloNode(vae3,
          optimizer3, 
          lossfn,
          parent=None, 
          children=[], 
          dataset = dataset3, 
          tuplesize=2, 
          batch_size=l3, 
          gammaManager = gammaManager_Independant(),
          Name="512"
    )

vaeR =  VAE(21, 5, dataset3.len_protein * dataset3.q, [512, 256, 128])
optimizerR = optim.Adam(vaeR.parameters(),weight_decay=0.01)
NodeR = PhyloNode(vaeR,
          optimizer3, 
          lossfn,
          parent=None, 
          children=[], 
          tuplesize=2, 
          batch_size=32, 
          gammaManager = gammaManager_Independant(),
          Name="Root"
    )


In [2]:
NodeR.addChildren(Node1)
NodeR.addChildren(Node2)
NodeR.addChildren(Node3)

In [6]:
recursive=True
NodeR.getNewTrainBatch(fullBatch=False)
NodeR.trainmode(recursive=recursive)
NodeR.computeLoss(recursive=recursive)
NodeR.computeCouplingLossChildren(recursive=recursive)

tensor(8561.5049, device='cuda:0', grad_fn=<AddBackward0>)

In [7]:
NodeR.model.encoder_linears[0].weight
NodeR.coupling_loss_Children

tensor(8561.5049, device='cuda:0', grad_fn=<AddBackward0>)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NodeR.to_(device, recursive=True)

In [ ]:
scheduler1 = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer1, factor=0.5, patience=10, verbose=True
)

In [ ]:
trainL = int(0.8 * len(dataset1))
testL = int(0.1 * len(dataset1))
valL = len(dataset1) - trainL -testL
batch_size = 64
train_set, test_set,  val_set = torch.utils.data.random_split(dataset1, [trainL, testL, valL])
train_iterator = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_iterator = DataLoader(test_set, batch_size=batch_size, shuffle=True)
val_iterator = DataLoader(val_set, batch_size=batch_size, shuffle=True)


In [ ]:
for epoch in range(100):
    print(epoch)
    vae1.train()
    train_loss_list = []
    for batch_idx, batch in enumerate(train_iterator):
        x,w = batch[0], batch[1]
        #targ = targ.unsqueeze(1)
        loss = -vae1.compute_weighted_elbo(x, w)
        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()
        train_loss_list.append(loss.item())
    
#         l2_reg = torch.tensor(0.)
#         for param in Dcla.parameters():
#             l2_reg += torch.norm(param)
#         loss += l2_lambda * l2_reg

    print(" acc train" ,np.mean(train_loss_list))
#     scheduler.step(np.mean(train_loss_list))
    test_loss_list = []
    with  torch.no_grad():
        for batch_idx, batch in enumerate(test_iterator):
            x,w = batch[0], batch[1]
            loss = -vae1.compute_weighted_elbo(x, w)
            test_loss_list.append(loss.item())
    print(" acc val" ,np.mean(test_loss_list))


In [ ]:
a = torch.tensor(1)
b = torch.tensor(2)
torch.min((a,2))

# kmeans Splitter

In [ ]:
dataset1.sequences.shape
len(dataset1)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=5, random_state=0).fit(dataset1.sequences)
kmeans.labels_
    #array([1, 1, 1, 0, 0, 0], dtype=int32)
# kmeans.predict([[0, 0], [12, 3]])
    #array([1, 0], dtype=int32)
kmeans.cluster_centers_
#     array([[10.,  2.],
#            [ 1.,  2.]])

In [ ]:
sum(kmeans.labels_==4)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(dataset1.sequences)

In [ ]:
import matplotlib.pyplot as plt
X_r = pca.transform(dataset1.sequences)
colors = ["navy", "turquoise", "darkorange", "red", "blue"]
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], kmeans.labels_):
    plt.scatter(
        X_r[kmeans.labels_ == i, 0], X_r[kmeans.labels_ == i, 1], color=color, alpha=0.8, lw=lw, label=target_name
    )
plt.legend(loc="best", shadow=False, scatterpoints=1)
plt.title("PCA of IRIS dataset")

plt.figure()

plt.show()

In [ ]:
sub = torch.utils.data.Subset(dataset1, kmeans.labels_==4)

In [ ]:
sub[:][0].shape

In [ ]:
a = torch.tensor(1)

In [ ]:
a.to(a.device)